In [6]:
import pandas as pd
import itertools
from sklearn.model_selection import KFold, cross_val_score
from deepforest import CascadeForestRegressor
import optuna
import numpy as np
np.int = int
np.bool = bool

In [7]:
def optuna_tune_deepforest(X, y, combo, n_trials=50):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 2, 10),  # forest 개수
            'max_layers': trial.suggest_int('max_layers', 2, 10),
            'n_trees': trial.suggest_int('n_trees', 100, 1000),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'n_jobs': -1,
            'random_state': 42
        }

        model = CascadeForestRegressor(**params)
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X[list(combo)], y, cv=kf, scoring='r2')
        return scores.mean()

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_trial.value, study.best_trial.params

def evaluate_and_tune_top_combos(X, y, top_n=3, n_trials=50):
    base_results = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    print("🔍 1단계: 전체 조합 성능 평가 중...\n")
    for r in range(1, len(X.columns) + 1):
        for combo in itertools.combinations(X.columns, r):
            X_subset = X[list(combo)]
            model = CascadeForestRegressor(n_estimators=2, n_trees=300, max_layers=5, random_state=42, n_jobs=-1)
            scores = cross_val_score(model, X_subset, y, cv=kf, scoring='r2')
            base_results.append({
                'features': combo,
                'mean_r2': scores.mean()
            })

    top_combos = sorted(base_results, key=lambda x: x['mean_r2'], reverse=True)[:top_n]

    print("⚙️ 2단계: 상위 조합에 대해 Optuna 하이퍼파라미터 튜닝 중...\n")

    for i, combo_info in enumerate(top_combos):
        combo = combo_info['features']
        print(f"⏳ {i+1}/{top_n} 조합 튜닝 중: {combo}")
        best_r2, best_params = optuna_tune_deepforest(X, y, combo, n_trials=n_trials)

        print(f"\n✅ 결과 {i+1}:")
        print(f"📌 Feature 조합: {combo}")
        print(f"📈 최적 R²: {best_r2:.4f}")
        print(f"⚙️ 최적 하이퍼파라미터: {best_params}")
        print("-" * 60)

# 데이터 불러오기
df = pd.read_csv('dataset.csv')
X = df[['cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age']]
y = df['CCS']

# 실행
evaluate_and_tune_top_combos(X, y, top_n=3, n_trials=50)

🔍 1단계: 전체 조합 성능 평가 중...

[2025-07-17 06:30:31.364] Start to fit the model:
[2025-07-17 06:30:31.364] Fitting cascade layer = 0 
[2025-07-17 06:30:33.314] layer = 0  | Val MSE = 198.50334 | Elapsed = 1.950 s
[2025-07-17 06:30:33.318] Fitting cascade layer = 1 
[2025-07-17 06:30:35.364] layer = 1  | Val MSE = 147.50555 | Elapsed = 2.046 s
[2025-07-17 06:30:35.369] Fitting cascade layer = 2 
[2025-07-17 06:30:37.701] layer = 2  | Val MSE = 151.63429 | Elapsed = 2.332 s
[2025-07-17 06:30:37.702] Early stopping counter: 1 out of 2
[2025-07-17 06:30:37.707] Fitting cascade layer = 3 
[2025-07-17 06:30:39.810] layer = 3  | Val MSE = 162.34517 | Elapsed = 2.103 s
[2025-07-17 06:30:39.810] Early stopping counter: 2 out of 2
[2025-07-17 06:30:39.810] Handling early stopping
[2025-07-17 06:30:39.814] The optimal number of layers: 2
[2025-07-17 06:30:39.821] Start to evalute the model:
[2025-07-17 06:30:39.821] Evaluating cascade layer = 0 
[2025-07-17 06:30:40.199] Evaluating cascade layer = 1 
[

[I 2025-07-17 18:45:20,121] A new study created in memory with name: no-name-7869a649-5b26-47a6-9b85-558bcf181da4


⚙️ 2단계: 상위 조합에 대해 Optuna 하이퍼파라미터 튜닝 중...

⏳ 1/3 조합 튜닝 중: ('cement', 'blast furnace slag', 'fly ash', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age')
[2025-07-17 09:45:20.138] Start to fit the model:
[2025-07-17 09:45:20.138] Fitting cascade layer = 0 
[2025-07-17 09:45:24.654] layer = 0  | Val MSE = 39.52659 | Elapsed = 4.517 s
[2025-07-17 09:45:24.662] Fitting cascade layer = 1 
[2025-07-17 09:45:28.593] layer = 1  | Val MSE = 23.82100 | Elapsed = 3.931 s
[2025-07-17 09:45:28.600] Fitting cascade layer = 2 
[2025-07-17 09:45:33.114] layer = 2  | Val MSE = 24.54237 | Elapsed = 4.514 s
[2025-07-17 09:45:33.115] Early stopping counter: 1 out of 2
[2025-07-17 09:45:33.123] Fitting cascade layer = 3 
[2025-07-17 09:45:37.130] layer = 3  | Val MSE = 26.66180 | Elapsed = 4.007 s
[2025-07-17 09:45:37.130] Early stopping counter: 2 out of 2
[2025-07-17 09:45:37.130] Handling early stopping
[2025-07-17 09:45:37.135] The optimal number of layers: 2
[2025-07-17 09:45:37

[I 2025-07-17 18:46:47,300] Trial 0 finished with value: 0.9136388120720543 and parameters: {'n_estimators': 3, 'max_layers': 8, 'n_trees': 367, 'min_samples_split': 8}. Best is trial 0 with value: 0.9136388120720543.


[2025-07-17 09:46:47.313] Start to fit the model:
[2025-07-17 09:46:47.313] Fitting cascade layer = 0 
[2025-07-17 09:47:05.130] layer = 0  | Val MSE = 36.19988 | Elapsed = 17.816 s
[2025-07-17 09:47:05.141] Fitting cascade layer = 1 
[2025-07-17 09:47:22.727] layer = 1  | Val MSE = 23.57064 | Elapsed = 17.586 s
[2025-07-17 09:47:22.737] Fitting cascade layer = 2 
[2025-07-17 09:47:40.277] layer = 2  | Val MSE = 24.97951 | Elapsed = 17.540 s
[2025-07-17 09:47:40.277] Early stopping counter: 1 out of 2
[2025-07-17 09:47:40.287] Fitting cascade layer = 3 
[2025-07-17 09:47:58.366] layer = 3  | Val MSE = 27.26637 | Elapsed = 18.079 s
[2025-07-17 09:47:58.366] Early stopping counter: 2 out of 2
[2025-07-17 09:47:58.366] Handling early stopping
[2025-07-17 09:47:58.383] The optimal number of layers: 2
[2025-07-17 09:47:58.408] Start to evalute the model:
[2025-07-17 09:47:58.409] Evaluating cascade layer = 0 
[2025-07-17 09:48:00.331] Evaluating cascade layer = 1 
[2025-07-17 09:48:02.623] 

...

[2025-07-17 16:13:45.338] Start to fit the model:
[2025-07-17 16:13:45.339] Fitting cascade layer = 0 
[2025-07-17 16:13:54.893] layer = 0  | Val MSE = 28.91503 | Elapsed = 9.555 s
[2025-07-17 16:13:54.908] Fitting cascade layer = 1 
[2025-07-17 16:14:04.301] layer = 1  | Val MSE = 22.63394 | Elapsed = 9.393 s
[2025-07-17 16:14:04.316] Fitting cascade layer = 2 
[2025-07-17 16:14:13.314] layer = 2  | Val MSE = 24.48764 | Elapsed = 8.998 s
[2025-07-17 16:14:13.315] Early stopping counter: 1 out of 2
[2025-07-17 16:14:13.315] Reaching the maximum number of layers: 3
[2025-07-17 16:14:13.316] Start to evalute the model:
[2025-07-17 16:14:13.316] Evaluating cascade layer = 0 
[2025-07-17 16:14:14.248] Evaluating cascade layer = 1 
[2025-07-17 16:14:15.329] Evaluating cascade layer = 2 
[2025-07-17 16:14:16.340] Start to fit the model:
[2025-07-17 16:14:16.341] Fitting cascade layer = 0 
[2025-07-17 16:14:26.272] layer = 0  | Val MSE = 26.45598 | Elapsed = 9.931 s
[2025-07-17 16:14:26.292] 

[I 2025-07-18 01:16:23,738] Trial 48 finished with value: 0.9152614932738125 and parameters: {'n_estimators': 8, 'max_layers': 3, 'n_trees': 314, 'min_samples_split': 3}. Best is trial 34 with value: 0.923068936303326.


[2025-07-17 16:16:23.814] Start to fit the model:
[2025-07-17 16:16:23.814] Fitting cascade layer = 0 
[2025-07-17 16:16:32.388] layer = 0  | Val MSE = 32.44116 | Elapsed = 8.575 s
[2025-07-17 16:16:32.404] Fitting cascade layer = 1 
[2025-07-17 16:16:41.189] layer = 1  | Val MSE = 22.69529 | Elapsed = 8.784 s
[2025-07-17 16:16:41.189] Reaching the maximum number of layers: 2
[2025-07-17 16:16:41.191] Start to evalute the model:
[2025-07-17 16:16:41.192] Evaluating cascade layer = 0 
[2025-07-17 16:16:42.323] Evaluating cascade layer = 1 
[2025-07-17 16:16:43.438] Start to fit the model:
[2025-07-17 16:16:43.438] Fitting cascade layer = 0 
[2025-07-17 16:16:51.917] layer = 0  | Val MSE = 29.87219 | Elapsed = 8.480 s
[2025-07-17 16:16:51.938] Fitting cascade layer = 1 
[2025-07-17 16:17:00.217] layer = 1  | Val MSE = 21.29339 | Elapsed = 8.279 s
[2025-07-17 16:17:00.217] Reaching the maximum number of layers: 2
[2025-07-17 16:17:00.218] Start to evalute the model:
[2025-07-17 16:17:00.2

[I 2025-07-18 01:17:50,713] Trial 49 finished with value: 0.9106909787732723 and parameters: {'n_estimators': 9, 'max_layers': 2, 'n_trees': 237, 'min_samples_split': 5}. Best is trial 34 with value: 0.923068936303326.



✅ 결과 3:
📌 Feature 조합: ('cement', 'blast furnace slag', 'water ', 'superplasticizer', 'coarse aggregate', 'fine aggregate', 'age')
📈 최적 R²: 0.9231
⚙️ 최적 하이퍼파라미터: {'n_estimators': 2, 'max_layers': 4, 'n_trees': 485, 'min_samples_split': 2}
------------------------------------------------------------
